synchronize your pydoxtools directory:

- rclone sync pydoxtools/ xyntopia_gdrive:/pydoxtools -P --size-only --fast-list

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
PDX_DIR="/content/pydoxtools"

In [3]:
%env TRAINING_DATA_DIR=/content/gdrive/MyDrive/pydoxtools/training_data

env: TRAINING_DATA_DIR=/content/gdrive/MyDrive/pydoxtools/training_data


In [4]:
!pip install faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.5 MB/s eta 0:00:00


In [5]:
!ls "/content/gdrive/My Drive/pydoxtools/pydoxtools"
!rm -r $PDX_DIR
!cp -r "/content/gdrive/My Drive/pydoxtools/pydoxtools" $PDX_DIR

classifier.py	    extract_tables.py	      ocr_language_mappings.py
class_utils.py	    extract_textstructure.py  pdf_utils.py
cluster_utils.py    file_utils.py	      __pycache__
document.py	    html_utils.py	      qamachine.py
extract_classes.py  initialize_library.py     random_data_generators.py
extract_files.py    __init__.py		      settings.py
extract_html.py     labeling.py		      training.py
extract_index.py    list_utils.py	      visual_document_analysis.py
extract_logic.py    math_utils.py	      webdav_utils.py
extract_objects.py  models.py
extract_spacy.py    nlp_utils.py
rm: cannot remove '/content/pydoxtools': No such file or directory


The main issue when using google colab for our training is, that we only have python version 3.9 available, while pydoxtools currently needs 3.10 to function.and

because of this we are trying to only use the relevant parts of the library which is mainly the data generation part which is used for
the training.

In [6]:
!rm $PDX_DIR/__init__.py
!touch $PDX_DIR/__init__.py
!pwd

/content


finally...  run pydoxtools

In [7]:
from pydoxtools import random_data_generators

## prepare training

In [8]:
bg = random_data_generators.TextBlockGenerator.std_generator()
bg.classmap, bg.classmap_inv, bg.num_generators, bg.class_gen, bg.gen_mapping, bg.weights

({0: 'address', 1: 'unknown'},
 {'address': 0, 'unknown': 1},
 3,
 (<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f125eba1a90>,
  <pydoxtools.random_data_generators.RandomListGenerator at 0x7f125eba1700>),
 {<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f125eba1a90>: 'address',
  <pydoxtools.random_data_generators.RandomTextBlockGenerator at 0x7f125eba1460>: 'unknown',
  <pydoxtools.random_data_generators.RandomListGenerator at 0x7f125eba1700>: 'unknown'},
 (100, 80, 20))

In [10]:
bg[10]

('66,"position":1,"preview_image":{"aspect_ratio":1.0,"height":1024,"width":1024,"src":"https:\\/\\/cdn.shopify.com\\/s\\/files\\/1\\/1896\\/8449\\/products\\/oreganowavesvelvetlongneutrals_65cmx30cm_front.png?v=1621352542"},"aspect_ratio":1.0,"height":1024,"media_type":"image","src":"https:\\/\\/cdn.shopify.com\\/s\\/files\\/1\\/1896\\/8449\\/products\\/oreganowa',
 tensor(1))

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.0 MB/s eta 0:00:00


In [33]:
from datasets import Dataset

dataset_size=10000

# generate datasets for finetuning
def my_gen():
    bgi = bg.__iter__()
    for i in range(1, dataset_size):
        text, label = next(bgi)
        yield {"label": label, "text":text}


dataset = Dataset.from_generator(my_gen)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-5a1f9e7f89abf550/0.0.0. Subsequent calls will reuse this data.


In [34]:
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 9999
})

In [35]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

In [36]:
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))

In [23]:
from transformers import AutoModelForSequenceClassification

In [24]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [15]:
from transformers import TrainingArguments

In [16]:
training_args = TrainingArguments(output_dir="test_trainer")
#training_args

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!pip install evaluate
import numpy as np
import evaluate

In [18]:
metric = evaluate.load("accuracy")

In [19]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [38]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_train_dataset,
    compute_metrics=compute_metrics,
)

In [43]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [39]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored